<a href="https://colab.research.google.com/github/Eassear/MyCode/blob/main/01_Game_SortTheMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 

In [ ]:
class Position(object):
  def __init__(self, i, j):
    if not(type(i) == type(1) or type(i) == type(1)):
      raise TypeError("position values must be int")
    self.i = i
    self.j = j
  def __str__(self):
    return "("+ str(self.i) +", "+str(self.j)+ ")"

In [ ]:
Rows = 3
Columns = 3
selected_size =(Rows, Columns)
board_items = [x for x in range(0, Rows*Columns)]

In [ ]:
board_items

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
goal = np.resize(board_items.copy(), selected_size)
goal

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [ ]:
def build_board():
  random_list = np.random.permutation(board_items)
  return random_list.reshape(selected_size)

In [ ]:
board = build_board()

In [ ]:
board

array([[7, 2, 6],
       [4, 5, 0],
       [1, 8, 3]])

In [ ]:
def possible_positions(matrix, position, route):
  a = position.i
  b = position.j
  d = {(c, r):(c, r) for c in range(Rows) for r in range(Columns)}
  possibles = filter(None, [d.get(e) for e in [(a, b+1), (a+1, b), (a-1, b), (a, b-1)]])
  l =  len(route)
  if l > 11:
    possibles = [x for x in possibles if not(route[l-12:l-6] == str(x))]
  return [Position(*x) for x in possibles]

In [ ]:
test_possible_positions = possible_positions(board, Position(1, 2), str((0, 2)))

In [ ]:
print(*test_possible_positions)

(2, 2) (0, 2) (1, 1)


In [ ]:
"(2, 2) (0, 2) (1, 1)"[14:20] == str((1, 1))

True

In [ ]:
def find_position_of_0(matrix):
  x, y = np.where(matrix == 0)
  return Position(int(x), int(y))

In [ ]:
class SimpleNode(object):
  def __init__ (self, matrix, value, route):
    self.matrix = matrix
    self.value = value
    self.route = route
  
  def __str__(self):
    return "("+ str(self.value)+", "+ self.route+ ")"

  def move_value(self, new_position, p0, matrix):
    return 1 + self.value
  
  def move(self, new_position, p0, matrix):
    matrix = np.copy(self.matrix)
    temp = matrix[new_position.i, new_position.j]
    matrix[new_position.i, new_position.j] = 0
    matrix[p0.i, p0.j] = temp
    value = self.move_value(new_position, p0, matrix)
    route = self.route+str(new_position)
    return matrix, value, route
  
  def create_childeren(self):
    children = []
    p0 = find_position_of_0(self.matrix)
    child_positions = possible_positions(self.matrix, p0, self.route)
    for p in child_positions:
      m_new, value, route = self.move(p, p0, self.matrix)
      children.append(SimpleNode(m_new, value, route))
    return children

In [ ]:
  node = SimpleNode(board,0, str(find_position_of_0(board)))
  agenda_first = np.array([node])
  print(*node.create_childeren())

(1, (1, 2)(2, 2)) (1, (1, 2)(0, 2)) (1, (1, 2)(1, 1))


In [ ]:
def recursive_search(agenda, t):
  min_node = agenda[0]
  if min_node.matrix.all == goal.all or t == 0:
    return agenda
  else:
    agenda = np.append(agenda, min_node.create_childeren()) 
    agenda.flat
    agenda = sorted(agenda,key=lambda x: x.value, reverse=False)
    agenda.pop(0)
    return recursive_search(agenda, t-1)

In [ ]:
results= recursive_search(agenda_first, 10)
for result in results:
  print(result)

(3, (1, 2)(2, 2)(2, 1)(2, 0))
(3, (1, 2)(0, 2)(0, 1)(1, 1))
(3, (1, 2)(0, 2)(0, 1)(0, 0))
(3, (1, 2)(1, 1)(2, 1)(2, 2))
(3, (1, 2)(1, 1)(2, 1)(2, 0))
(3, (1, 2)(1, 1)(0, 1)(0, 2))
(3, (1, 2)(1, 1)(0, 1)(0, 0))
(3, (1, 2)(1, 1)(1, 0)(2, 0))
(3, (1, 2)(1, 1)(1, 0)(0, 0))
(4, (1, 2)(2, 2)(2, 1)(1, 1)(1, 2))
(4, (1, 2)(2, 2)(2, 1)(1, 1)(0, 1))
(4, (1, 2)(2, 2)(2, 1)(1, 1)(1, 0))


In [ ]:
class Right_Move_First(SimpleNode):
  def move_value(self, new_position, p0, matrix):
    total_difference = 0
    Row, Col = np.shape(goal)
    for i in range(Row):
      for j in range(Col):
        x, y = np.where(matrix ==goal[i, j])
        total_difference += pow(pow(int(x)-i, 2) + pow(int(y)-j, 2), 0.5)
    return self.value + total_difference/10

In [ ]:
  node = Right_Move_First(board,1, str(find_position_of_0(board)))
  agenda_first = np.array([node])
  results= recursive_search(agenda_first, 100)
  for result in results:
    print(result)

(7.612899020449196, (1, 2)(2, 2)(2, 1)(1, 1)(1, 0)(0, 0)(0, 1))
(7.612899020449196, (1, 2)(2, 2)(2, 1)(2, 0)(1, 0)(1, 1)(1, 2))
(7.612899020449196, (1, 2)(2, 2)(2, 1)(2, 0)(1, 0)(1, 1)(2, 1))
(7.612899020449196, (1, 2)(2, 2)(2, 1)(2, 0)(1, 0)(1, 1)(0, 1))
(7.612899020449196, (1, 2)(2, 2)(2, 1)(2, 0)(1, 0)(0, 0)(0, 1))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(0, 2)(0, 1)(1, 1))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(0, 2)(0, 1)(0, 0))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(1, 1)(2, 1)(2, 2))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(1, 1)(2, 1)(2, 0))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(1, 1)(0, 1)(0, 2))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(1, 1)(0, 1)(0, 0))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(1, 1)(1, 0)(2, 0))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 2)(1, 2)(1, 1)(1, 0)(0, 0))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 0)(1, 0)(1, 1)(1, 2)(2, 2))
(8.395084461961865, (1, 2)(1, 1)(2, 1)(2, 0)(1, 0)